# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

Let's start by setting up a basic agent to demonstrate different streaming approaches.


In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    systemPrompt: "You are a full-stack comedian",
})

Now that we have our agent configured, let's first see how it works without streaming.


## No Steaming (invoke)

In [ ]:
import { HumanMessage } from "langchain";

const result = await agent.invoke({
    messages: [new HumanMessage("Tell me a joke")]
})

console.log(result.messages.at(-1).content)

Notice how we had to wait for the complete response. Let's improve the user experience with streaming.


## Streaming
### `value`

In [ ]:
const stream = await agent.stream({
    messages: [new HumanMessage("Tell me a joke")],
}, {
    streamMode: "values",
})

for await (const step of stream) {
    console.log(step.messages.at(-1).content)
}

The `values` mode streams the entire state at each step. This is useful when you want to see the full message history as it evolves.

### `messages`

For a more granular experience, `messages` mode streams individual message chunks as they're generated - perfect for real-time token-by-token display.


In [ ]:
const stream = await agent.stream({
    messages: [new HumanMessage("Tell me a joke")],
}, {
    streamMode: "messages",
})

for await (const [message, metadata] of stream) {
    console.log(`[${metadata.langgraph_node}]: ${message.content}`)
}

Here's a more dramatic example - streaming a poem token by token creates a typewriter effect:


In [ ]:
await Deno.jupyter.broadcast("display_data", {
    data: { "text/markdown": "🤔" },
    metadata: {},
    transient: { display_id: "progress" }
});

const stream = await agent.stream({
    messages: [new HumanMessage("Write me a poem.")],
}, {
    streamMode: "messages",
})

let content = "";
const i = setInterval(async () => {
    Deno.jupyter.broadcast("update_display_data", {
        data: { "text/markdown": content },
        metadata: {},
        transient: { display_id: "progress" }
    });
}, 1)

for await (const [message, metadata] of stream) {
    content += message.content
}

clearInterval(i)

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A stream writer allows you to easily stream `custom` data from sources you create.

In [ ]:
import * as setup from "./setup.ts";
import { z } from "zod";
import { createAgent, tool, type Runtime } from "langchain";

const getWeather = tool(({ city }, runtime: Runtime) => {
    runtime.writer(`Looking up data for city: ${city}`);
    runtime.writer(`Acquired data for city: ${city}`);
    return `It's always sunny in ${city}`
}, {
    name: "get_weather",
    description: "Get weather for a given city.",
    schema: z.object({
        city: z.string()
    })
});

const toolCallingAgent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [getWeather]
})

const stream = await toolCallingAgent.stream({
    messages: "What's the weather in SF?",
}, {
    streamMode: ["values", "custom"],
})

for await (const [type, stateOrCustomEvent] of stream) {
    if (type === "values") {
        displayMessage(stateOrCustomEvent.messages.at(-1))
    } else if (type === "custom") {
        displayMessage({
            type,
            content: stateOrCustomEvent
        })
    }
}


┌────────────────────────────────────────────────────────────┐
│ 👤 HUMAN MESSAGE                                           │
└────────────────────────────────────────────────────────────┘
What's the weather in SF?

┌────────────────────────────────────────────────────────────┐
│ 🤖 AI MESSAGE                                              │
└────────────────────────────────────────────────────────────┘
[
  {
    type: "tool_use",
    id: "toolu_01QgYgp7Ux4R8mfthYDgt3Rn",
    name: "get_weather",
    input: { city: "San Francisco" }
  }
]

┌────────────────────────────────────────────────────────────┐
│ 🔧 TOOL MESSAGE                                            │
└────────────────────────────────────────────────────────────┘
It's always sunny in San Francisco

┌────────────────────────────────────────────────────────────┐
│ 🤖 AI MESSAGE                                              │
└────────────────────────────────────────────────────────────┘
The weather in San Francisco is sunny!


## Try your own.
Create a tool of your own and try it here!

In [ ]:
const stream = await agent.stream({
    messages: "What's the weather in SF?",
}, {
    streamMode: ["values", "custom"],
})

for await (const [type, chunk] of stream) {
    if (type === "custom") {
        console.log(chunk)
    }
}